<a href="https://colab.research.google.com/github/roberthouston14/General/blob/main/Bert_Sentiment_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.6 MB/s eta 0:00:00


In [3]:
# Prompt the user for the file path
csv_file_path = input('Enter the path to your CSV file in Google Drive: ')

Enter the path to your CSV file in Google Drive: /content/gdrive/MyDrive/Production Datasets/Working_Gold_Data_POS_Features_Spellcheck_Features.csv


In [7]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

# Read the CSV file
data = pd.read_csv(csv_file_path)

# Randomly sample 20 rows from the DataFrame
data = data.sample(n=20, random_state=42)

# Extract the 'text' column
texts = data['text'].tolist()

# Load a tokenizer for your transformer model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Convert the inputs to a PyTorch dataset
text_dataset = TextDataset(inputs)

# Create a DataLoader
batch_size = 8
dataloader = DataLoader(text_dataset, batch_size=batch_size)

# Load a transformer model
model = AutoModel.from_pretrained('bert-base-uncased')

# Extract features using the transformer model
features = []
model.eval()

for batch in dataloader:
    with torch.no_grad():
        output = model(**batch)
        # Append extracted features to the features list
        features.extend(output.last_hidden_state[:, 0, :].cpu().numpy())

# Add a new column "sentiment" to the DataFrame
data['sentiment'] = features

# Save the updated DataFrame to a new CSV file
output_file_path = os.path.splitext(csv_file_path)[0] + '_sentiment.csv'
data.to_csv(output_file_path, index=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-7-182d1b129983>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo